### How to extract data from spotify using spotify api

In [2]:
!pip install python-dotenv spotipy
!pip install spotipy
import csv
import os
import re
from dotenv import load_dotenv
import pandas as pd
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import base64
from requests import post 
import json 

In [3]:
load_dotenv()

client_id = os.getenv("CLIENT_ID","0624d92123024fd4a7b2017c8d531c94") # tyoe in your client_id in between the quotation marks
client_secret = os.getenv("CLIENT_SECRET", "4127bf4ec5cf4ab28140a7bba2036ac9") # input your client_secret in between the quotation marks

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret= client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
link = "https://open.spotify.com/playlist/37i9dQZF1DX2Zjg8kBuEsQ"
uris = link.split("/")[-1]
track_uris = [x['track']['uri'] for x in sp.playlist_tracks(uris)['items']]

In [9]:
output_file = "track_info.csv"

with open(output_file, "w", encoding = 'utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['track_uri', 'track', 'artist', 'artist_popularity', 'followers', 'artist_genre', 'track_popularity', 'album'])
        for track in sp.playlist_tracks(uris)["items"]:
            #URI
            track_uri = track["track"]["uri"]
            
            #Track name
            track_name = track["track"]["name"]
            
            #Main Artist
            artist_uri = track["track"]["artists"][0]["uri"]
            artist_info = sp.artist(artist_uri)
            
            #Name, popularity, genre
            artist_name = track["track"]["artists"][0]["name"]
            artist_pop = artist_info["popularity"]
            artist_genre = artist_info["genres"]
            followers = artist_info["followers"]['total']

            #Album
            album = track["track"]["album"]["name"]
            
            #Popularity of the track
            track_pop = track["track"]["popularity"]
            
            # write a csv file
            writer.writerow([track_uri, track_name, artist_name, artist_pop, followers, artist_genre, track_pop, album])

In [10]:
df = pd.read_csv('track_info.csv')
df.head()

,track_uri,track,artist,artist_popularity,followers,artist_genre,track_popularity,album
0,spotify:track:4a3D8glGZxhlJfOs1Czc3c,About Damn Time,Lizzo,79,5263886,"['dance pop', 'escape room', 'minnesota hip ho...",60,Special
1,spotify:track:3LtpKP5abr2qqjunvjlX5i,Doja,Central Cee,83,2131416,['melodic drill'],89,Doja
2,spotify:track:39JofJHEtg8I4fSyo7Imft,B.O.T.A. (Baddest Of Them All) - Edit,Eliza Rose,70,26122,['house'],85,B.O.T.A. (Baddest Of Them All)
3,spotify:track:0wHFktze2PHC5jDt3B17DC,First Class,Jack Harlow,82,2743582,"['deep underground hip hop', 'kentucky hip hop...",84,Come Home The Kids Miss You
4,spotify:track:29d0nY7TzCoi22XBqDQkiP,Running Up That Hill (A Deal With God) - 2018 ...,Kate Bush,74,1578660,"['art pop', 'art rock', 'baroque pop', 'new wa...",80,Hounds of Love (2018 Remaster)


In [11]:
new = df['track_uri'].str.split(":", expand = True)
df['track_uri'] = new[2].astype(str)

In [12]:
def get_token():
    '''Get token'''
    auth_string = client_id + ":" + client_secret 
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {'grant_type': "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()

headers = {'Authorization': 'Bearer {}'.format(token)}

In [13]:
base_url = 'https://api.spotify.com/v1/'

uris = df['track_uri'].to_list()

features = {}

with open("track_features.csv", "w", encoding = 'utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["track_uri", "danceability", "energy", "speechiness", "instrumentalness", "tempo"])
        for uri in uris:
            track_uri = uri
            features[uri] = {
                            'danceability': 0,
                            'energy': 0,
                            'speechiness': 0, 
                            'instrumentalness': 0,
                            'tempo': 0
                            }

            s = requests.get(base_url + 'audio-features/' + uri, headers = headers)
            s = s.json()
            
            features[uri]['danceability'] = s['danceability']
            features[uri]['energy'] = s['energy']
            features[uri]['speechiness'] = s['speechiness']
            features[uri]['instrumentalness'] = s['instrumentalness']
            features[uri]['tempo'] = s['tempo']
            writer.writerow([track_uri, features[uri]['danceability'], features[uri]['energy'], features[uri]['speechiness'], 
                            features[uri]['instrumentalness'], features[uri]['tempo']])

In [14]:
df2 = pd.read_csv("track_features.csv")
df2.head(2)

,track_uri,danceability,energy,speechiness,instrumentalness,tempo
0,4a3D8glGZxhlJfOs1Czc3c,0.835,0.747,0.0622,0.0,108.974
1,3LtpKP5abr2qqjunvjlX5i,0.911,0.573,0.2880,0.0,140.040


In [15]:
df = pd.merge(df, df2, how = 'inner', on = 'track_uri')
df.head(1)

,track_uri,track,artist,artist_popularity,followers,artist_genre,track_popularity,album,danceability,energy,speechiness,instrumentalness,tempo
0,4a3D8glGZxhlJfOs1Czc3c,About Damn Time,Lizzo,79,5263886,"['dance pop', 'escape room', 'minnesota hip ho...",60,Special,0.835,0.747,0.0622,0.0,108.974


In [16]:
# Save the .csv file 
df.to_csv('tracks.csv', index = False)